<a href="https://colab.research.google.com/github/JoelDoerfel/Generative_AI/blob/main/Langchain_QS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install requirements
!pip install langchain --quiet
print('langchain installed')
!pip install openai --quiet
print('openai installed')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.0/759.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.0 MB/s eta 0:00:00
langchain installed
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.1 MB/s eta 0:00:00
openai installed


In [30]:
# import secrets
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
with open('/content/drive/My Drive/credentials.json') as f:
    text = f.read()

import json
secrets = json.loads(text)
openai_api_key = secrets['OPENAI_API_KEY']
print('secrets imported')

secrets imported


In [29]:
# bring data inputs into dataframe
import pandas as pd

url = 'https://raw.githubusercontent.com/JoelDoerfel/Generative_AI/main/Custom_Dataframe/events.csv'
df = pd.read_csv(url)
df['date'] = df['date'].astype({'date': 'datetime64[ns]'})
df

,date,wake,sleep,sleep_hours,mood,body_fat,am_weight_lbs,calories_in,calories_out,net_calories,fitbit_steps
0,2020-01-01,7:30 AM,11:11 PM,8.0,70,29.40%,224.6,1800,560,-1455,14000
1,2020-01-02,7:30 AM,11:11 PM,8.3,72,30.60%,223.2,2700,240,-218,6000
2,2020-01-03,6:20 AM,9:30 PM,7.2,72,29.60%,221.7,2000,440,-1100,11000
3,2020-01-04,7:00 AM,10:10 PM,9.5,80,30.10%,222.8,2000,520,-1194,13000
4,2020-01-05,6:40 AM,12:12 AM,8.5,80,28.80%,219.4,1800,320,-1153,8000
...,...,...,...,...,...,...,...,...,...,...,...
1090,2022-12-27,5:21 AM,9:45 PM,8.3,72,24.50%,204.5,2100,620,-974,5000
1091,2022-12-28,5:45 AM,9:09 PM,8.0,70,24.80%,204.2,1400,150,-1201,3000
1092,2022-12-29,5:45 AM,10:15 PM,8.6,70,26.00%,203.1,1700,150,-887,3000
1093,2022-12-30,6:30 AM,9:30 PM,8.3,70,25.80%,202.2,1600,150,-976,3000


In [32]:
# build the langchain agent
from langchain.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI
import os
os.environ['OPENAI_API_KEY'] = openai_api_key

agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

# ask questions
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to find the number of rows
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 1095
Thought: I now know the final answer
Final Answer: 1095 rows

> Finished chain.


'1095 rows'

In [33]:
agent.run("what was the average am_weight_lbs in when the year of the date column was 2020?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows with a date in 2020
Action: python_repl_ast
Action Input: df[df['date'].dt.year == 2020]['am_weight_lbs'].mean()
Observation: 212.28688524590163
Thought: I now know the final answer
Final Answer: The average am_weight_lbs in 2020 was 212.29.

> Finished chain.


'The average am_weight_lbs in 2020 was 212.29.'